<h1>    FACE MASK DETECTION

<h1>Importing all  important Libraries

In [ ]:
import numpy as np 
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf

import os
from keras.preprocessing import image
import random  # to shuffle the data in data list that has values with and without mask in a linear manner




# lets do transfer learning from vgg16 hence we will import vgg and related libraries
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input,Flatten,Dense
from tensorflow.keras.models import Model


In [ ]:
categories=["with_mask","without_mask"] # categories of images

<h1>Loading the Dataset and storing in a list

In [ ]:
data=[] # list to store the images and labels


for category in categories:                                                        # loop to iterate through the categories
  path=os.path.join(r"C:\Users\viswa\OneDrive\Desktop\facemask_dataset",category)  # path to the folder containing the images
  print(path)       
  # print(0)
  label=categories.index(category)                                              # label to be assigned to the images

  for file in os.listdir(path):                                                # loop to iterate through the images in the folder
    img_path=os.path.join(path,file)
    img=cv2.imread(img_path)
    img=cv2.resize(img,(224,224))                                              # resizing the image to 224*224

    
    data.append([img,label])                                                 # appending the image and label to the data list

# path



In [ ]:
print(len(data)) # length of the data list
data[0][0].shape # shape of the image

<h1>shuffling the data and separating the input and target features

In [ ]:
import random   # to shuffle the data in data list that has values with and without mask in a linear manner
random.shuffle(data) # shuffling the data

X=[]    # list to store the images
y=[]    # list to store the labels
for features,label in data:         # loop to iterate through the data list
    X.append(features)
    y.append(label)

X=np.array(X)               # converting the list to numpy array
y=np.array(y)                   # converting the list to numpy array


In [ ]:
X.shape # shape of the image

In [ ]:
y.shape         # shape of the label

In [ ]:
X=X/255.0       # normalizing the image



<h1>splitting the data in to train test samples 

In [ ]:
# lets split the data into train and test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)           # splitting the data into train and test with test size of 20%


In [ ]:
X_train.shape    # shape of the train data

In [ ]:
X_test.shape        # shape of the test data

In [ ]:
# lets do transfer learning from vg16
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input,Flatten,Dense
from tensorflow.keras.models import Model

<h1>Transfer learning with VGG16


In [ ]:
vgg=VGG16()   # importing the vgg16 model
vgg.summary()  # summary of the model


In [ ]:
from keras import Sequential    # to add layers one by one
model=Sequential()        # creating a sequential model

In [ ]:
for layer in vgg.layers[:-1]:   # loop to iterate through the layers of vgg16 model
    model.add(layer)            # adding the layers to the model

model.summary()     # summary of the model

In [ ]:
for layer in model.layers:              # loop to iterate through the layers of the model

    layer.trainable=False               # making the layers of the model non trainable(i.e freezing the layers)

In [ ]:
model.summary()             # summary of the model after freezing the layers

In [ ]:
# add last layer of our data to vgg model
model.add(Dense(1,activation='sigmoid'))   # adding the last layer to the model with sigmoid activation function
model.summary()                             # summary of the model after adding the last layer to the vgg model with sigmoid activation function

In [ ]:
# compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])  # compiling the model with adam optimizer and binary crossentropy as loss function and accuracy as metrics

<H1>Fitting the data and checking the accuracy

In [ ]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

<h1>we got accuracy of 0.94 in train and 0.93 in validation test

In [ ]:
# lets use opencv to detect the face and mask in video
import cv2                  # importing opencv library

import numpy as np
from keras.preprocessing import image     # importing image from keras preprocessing library




In [ ]:
def detect_face_mask(img):                  # function to detect the face mask
    
    y_pred=model.predict(img.reshape(1,224,224,3))    # predicting the image
    return y_pred[0][0]                                 # returning the prediction
    # return img.shape


In [ ]:
def predict_classess(y_pred):               # function to predict the classes
    
    if y_pred<0.5:
        print("with mask")
    else:
        print("without mask")

In [ ]:
def draw_label(img,text,pos,bg_color):          # function to draw the label

    text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x=pos[0]+text_size[0][0]+2
    end_y=pos[1]+text_size[0][1]-2

    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)


<h1>motion capturing to detect face mask on moving people

In [ ]:
cap=cv2.VideoCapture(0)     # capturing the video from the webcam

In [ ]:


while True:                             # loop to iterate through the video

    ret,frame=cap.read()                        # ret will return true if the frame is available

    img=cv2.resize(frame,(224,224))             # resizing the image to 224*224
    y_predict=detect_face_mask(img)             # predicting the image
    # print(y_predict)

    # predict_classess(y_predict)


    if y_predict<=0.5:                             # if the prediction is less than 0.5 then the person is wearing mask
        draw_label(frame,"mask",(20,20),(0,255,0))
    else:
        draw_label(frame,"no mask",(20,20),(0,0,255))
    


    cv2.imshow("window",frame)
    
    
    if cv2.waitKey(1)&0xFF==ord('x'):
        break
        # video can break by pressing x key


xxx



cv2.destroyAllWindows()









